## This notebook produces cases per year for the benchmark simulation

In [1]:
import numpy as np
import pandas as pd
import itertools
import pickle
import datetime, random, copy
import sys, os, tqdm

import SEIR_full as mdl
import SEIR_full.model as mdl
import SEIR_full.calibration as mdl
from SEIR_full.parameters import *
from SEIR_full.indices import *
from SEIR_full.utils import *

* Fixing the ranges of the uncertainty factors that we're interested in:

In [2]:
# Creating a list of tuples, each tuple is a realization of randomly picked values for the factors that we change every iteration of the simulation
np.random.seed(42)
list_of_tuples = []
for i in range(1000):
    booster_eff = round(np.random.uniform(low=29.5001, high=95.5)) / 100     # Discrete uniform dist, booster_efficiency ~ U(30, 95), Updated in the model's object
    inv_level = round(np.random.uniform(low=9.5001, high=20.5)) / 100       # Discrete uniform dist, booster_efficiency ~ U(10, 20), Updated in the simulation's settings
    hosp_proba_scalar = round(np.random.uniform(low=0.8, high=1.2), 2)      # Continuous uniform dist, booster_efficiency ~ U(0.8, 1.2), Updated in the model's object
    years_for_model_run = round(np.random.uniform(low=.5001, high=5.5))         # Discrete uniform dist, booster_efficiency ~ U(3, 10), Updated in the simulation's settings
    list_of_tuples.append((booster_eff, inv_level, hosp_proba_scalar, years_for_model_run))

* Fixing the static settings for all runs of the big loop

In [3]:
## DEFINING STATIC SETTINGS (INDEPENDENT IN THE DYNAMIC FACTORS) FOR THE SIMULATION RUN
# Short path for the data directory
DATA_DIR = r'/Users/yotamdery/Old_Desktop/git/SEIR_model_COVID-main/Data'
# Reading the indices of the model - adding an hint to declare it's from type Indices!
with(open(DATA_DIR + '/parameters/indices.pickle', 'rb')) as openfile:
    ind:Indices = pickle.load(openfile)

# Vaccination priority queue:
vaccination_pq = [('High', '60+'), ('High', '30-59'), ('High', '10-29'), ('High', '5-9'),
                   ('Low', '60+'), ('Low', '30-59'), ('Low', '10-29'), ('Low', '5-9')]

# Reading the neutralized vectors:
# Alpha variant vector
with open(DATA_DIR + '/parameters/neutralized_alpha_variant_vec.pickle', 'rb') as pickle_in:
	neutralized_alpha_variant_vec = pickle.load(pickle_in)

# Delta variant vector
with open(DATA_DIR + '/parameters/neutralized_delta_variant_vec.pickle', 'rb') as pickle_in:
	neutralized_delta_variant_vec = pickle.load(pickle_in)

# Beta_lockdown vector
with open(DATA_DIR + '/parameters/neutralized_lockdown_vec.pickle', 'rb') as pickle_in:
	neutralized_lockdown_vec = pickle.load(pickle_in)

# Beta_school vector
with open(DATA_DIR + '/parameters/neutralized_school_vec.pickle', 'rb') as pickle_in:
	neutralized_school_vec = pickle.load(pickle_in)

# Isolation morbidity ratio vector
with open(DATA_DIR + '/parameters/neutralized_isolation_morbidity_ratio_vector.pickle', 'rb') as pickle_in:
	neutralized_isolation_morbidity_ratio_vector = pickle.load(pickle_in)

# zero vector to remove any transition from V_2 and S_2 to V_3
with open(DATA_DIR + '/parameters/neutralized_transition_rate_to_V_3.pickle', 'rb') as pickle_in:
	neutralized_transition_vector = pickle.load(pickle_in)


* Utils functions:

In [4]:
# Getting the reported unreported ratio
def get_reported_unreported_ratio(scen):
    if scen == 'Scenario2':
        reported = 1
        unreported = 2
        reported_unreported = unreported / (reported + unreported)
    return reported_unreported

def risk_age_groups_mapping(wanted_mapping: tuple):
    """ This function gets a wanted mapping (4 examined age groups to 9 original model's age groups or vice versa),
    and returns a dictionary consist of the wanted mapping
    """
    age_groups_mapping_dict = {}
    # If the wanted mapping is 4 examined age groups to 9 original model's age groups:
    if wanted_mapping == ('High', '4-to-9'):
        age_groups_mapping_dict[('High', '5-9')] = [('High', '5-9')]
        age_groups_mapping_dict[('High', '10-29')] = [('High', '10-19'), ('High', '20-29')]
        age_groups_mapping_dict[('High', '30-59')] = [('High','30-39'), ('High','40-49'), ('High','50-59')]
        age_groups_mapping_dict[('High', '60+')] = [('High', '60-69'), ('High', '70+')]

    elif wanted_mapping == ('Low', '4-to-9'):
        age_groups_mapping_dict[('Low', '5-9')] = [('Low', '5-9')]
        age_groups_mapping_dict[('Low', '10-29')] = [('Low', '10-19'), ('Low', '20-29')]
        age_groups_mapping_dict[('Low', '30-59')] = [('Low','30-39'), ('Low','40-49'), ('Low','50-59')]
        age_groups_mapping_dict[('Low', '60+')] = [('Low', '60-69'), ('Low', '70+')]

    # If the wanted mapping is 9 original model's age groups to 4 examined age groups:
    elif wanted_mapping == ('High', '9-to-4'):
        age_groups_mapping_dict[('High', '0-4')] = np.nan     # There's no corresponding age group among the 4 examined age-groups
        age_groups_mapping_dict[('High', '5-9')] = ('High', '5-9')
        age_groups_mapping_dict[('High', '10-19')] = ('High', '10-29')
        age_groups_mapping_dict[('High', '20-29')] = ('High', '10-29')
        age_groups_mapping_dict[('High', '30-39')] = ('High', '30-59')
        age_groups_mapping_dict[('High', '40-49')] = ('High', '30-59')
        age_groups_mapping_dict[('High', '50-59')] = ('High', '30-59')
        age_groups_mapping_dict[('High', '60-69')] = ('High', '60+')
        age_groups_mapping_dict[('High', '70+')] = ('High', '60+')

    else:
        age_groups_mapping_dict[('low', '0-4')] = np.nan     # There's no corresponding age group among the 4 examined age-groups
        age_groups_mapping_dict[('low', '5-9')] = ('low', '5-9')
        age_groups_mapping_dict[('low', '10-19')] = ('low', '10-29')
        age_groups_mapping_dict[('low', '20-29')] = ('low', '10-29')
        age_groups_mapping_dict[('low', '30-39')] = ('low', '30-59')
        age_groups_mapping_dict[('low', '40-49')] = ('low', '30-59')
        age_groups_mapping_dict[('low', '50-59')] = ('low', '30-59')
        age_groups_mapping_dict[('low', '60-69')] = ('low', '60+')
        age_groups_mapping_dict[('low', '70+')] = ('low', '60+')

    return age_groups_mapping_dict

def get_wanted_df(wanted_age_group):
	""" Gets the age-groups that we want to address our calculations to, and returns a sub-Dataframe containing only the relevant columns (the wanted age-groups columns)"""
	# Reading the pop per county file:
	county_pop = pd.read_csv(DATA_DIR + '/division_choice/30/population_per_county_age-group.csv')
	# Initializing the list with the county column, that will be picked anyway
	wanted_columns = [county_pop.columns[0]]
	# Splitting the given age_group, creating list of integers. try-expect block to catch the 60+ column as well:
	try:
		wanted_age_group_splitted = list(map(int, wanted_age_group.split('-')))
	except ValueError:
		wanted_age_group_splitted = [60, 70]  # In order to pass the logic condition successfully

	# Iterating on the columns of the file and performing the condition logic:
	for col in county_pop.columns[1:]:
		# Splitting the current column of the file. try-expect block to catch the 70+ column as well:
		try:
			ages_as_list = list(map(int, col.split('_')[1].split('-')))
		except ValueError:
			ages_as_list = [70]
		# If these conditions are met, take this column to be in the sub-df
		if ages_as_list[0] >= min(wanted_age_group_splitted) and ages_as_list[0] <= max(wanted_age_group_splitted):
			wanted_columns.append(col)

	return county_pop[wanted_columns]

# Gets a dictionary to reduce (lambda, V_2 or S_2) and reduce its cardinality to consist only the relevant age-groups
def calc_sub_dict(dict_to_reduce: dict, wanted_age_group: str):
    sub_dict = {}
    # Splitting the given age_group, creating list of integers. try-expect block to catch the 60+ group as well (if wanted):
    try:
        wanted_age_group_splitted = list(map(int, wanted_age_group.split('-')))
    except ValueError:
        wanted_age_group_splitted = [60, 70]    # In order to pass the logic condition successfully

    for key, val in dict_to_reduce.items():
        # Splitting the age group part of the key. try-expect block to catch the 70+ column from the file as well:
        try:
            if isinstance(key, str):   # If the dict_to_reduce keys are only on the age level
                ages_as_list = list(map(int, key.split('-')))
            elif isinstance(key, tuple):    # If the dict_to_reduce keys are on the region&age level (as lambda_t does) - take the second element (the age)
                ages_as_list = list(map(int, key[1].split('-')))
        except ValueError:
            ages_as_list = [70]
        # If these conditions are met, take this key-value pair to be in the sub-dict
        if ages_as_list[0] >= min(wanted_age_group_splitted) and ages_as_list[0] <= max(wanted_age_group_splitted):
            sub_dict[key] = val    # Converting from ndarray to float

    return sub_dict

def get_indexes_risk_age_combination(risk_age_group: tuple):
    """This function gets a tuple of (risk, 4_age_group) and returns the indexes of that combination as they are in the original model (including transformation to the 9 original age groups
    e.g.: for ('High', '10-29'), returns the indexes for ('High', '10-19') + ('High', '20-29') """
    # Initializing the result list and the age groups mapper:
    indexes_list_of_lists = []
    four_to_nine_age_map = risk_age_groups_mapping((risk_age_group[0], '4-to-9'))
    for val in four_to_nine_age_map[risk_age_group]:
            indexes_list_of_lists.append(ind.risk_age_dict[val])
    # Merging the list of lists to one list:
    risk_indexes_list = [item for sublist in indexes_list_of_lists for item in sublist]
    return sorted(risk_indexes_list)

# Gets the compartment to aggregate, and returns the same compartment, stratified by the original age-groups
def calc_aggregated_compartment_by_age_risk(compartment: np.array):
    compartment_dict = {}
    for risk_age in ind.risk_age_dict.keys():
        compartment_dict[risk_age] = float(compartment[:, ind.risk_age_dict[risk_age]].sum(axis=1))
    return compartment_dict

# Gets the compartment dictionary (the compartment stratified by the old age-groups), and aggregates the compartment to the wanted age-group by returning a dict (e.g agg. V_2 for age group 30-59, using the sub-age-groups 30-39, 40-49, 50-59)
def compartment_by_4_age_groups(compartment: dict, target_risk_age_group: tuple):
    # Initiate the wanted mapping:
    four_to_nine_age_map = risk_age_groups_mapping((target_risk_age_group[0], '4-to-9'))
    # Initiate result dict to return:
    agg_risk_4_groups_dict = {}
    for risk_4_age_group in four_to_nine_age_map.keys():     # Iterating on each combination of (risk, 9 age-groups):
        agg_risk_4_groups_dict[risk_4_age_group] = 0
    # Performing the aggregation to the new dict - (risk, 4 age groups):
    for key, val in four_to_nine_age_map.items():
        for tup in val:
            agg_risk_4_groups_dict[key] += compartment[tup]

    return agg_risk_4_groups_dict

def calc_comp_prop_for_age_risk(comp: str, curr_res_mdl_1_ml: dict, target_risk_age_group: tuple):
    """This function gets the target compartment and the current predictions of the current model,
    and returns the compartment aggregated by the wanted combination of (age-group, risk)"""
    # Getting the last day of the compartment:
    comp_last_day = np.reshape(curr_res_mdl_1_ml[comp][-1, :], newshape=(1,540))
    # Aggregate the compartment by (9 age groups, risk):
    comp_agg_9_age_groups_risk = calc_aggregated_compartment_by_age_risk(comp_last_day)
    # Aggregate the compartment by (4 age groups, risk):
    comp_agg_4_age_groups_risk = compartment_by_4_age_groups(comp_agg_9_age_groups_risk, target_risk_age_group)
    # Gets the relevant age group out of the 4 age groups, and summing on it to get the final proportion:
    return comp_agg_4_age_groups_risk[target_risk_age_group]

def update_compartment(curr_res_mdl_1_ml: dict, compartment: str, updated_array: np.array):
    """This function gets the current models' predictions, the compartment to update and the updated array, and updates the model object with the updated array"""
    # Initiate settings:
    curr_res_mdl_1_ml[compartment][-1] = updated_array
    comp_as_list = []
    # Changing the updated arrays to be lists of arrays (to allow the update of the model object):
    for element in curr_res_mdl_1_ml[compartment]:
        comp_as_list.append(element)

    # Performing the proper model update
    curr_res_mdl_1_ml.update({
        compartment : comp_as_list
    })

# Defining the model - initializing it for t=0:
def defining_model(scen):
    model_1_ml_ = mdl.Model_behave(
    ind= ind,
    beta_j= cal_parameters[scen]['beta_j'],
    beta_activity= cal_parameters[scen]['beta_activity'],
    beta_school= cal_parameters[scen]['beta school'],
    scen= scen
    )
    # Predicting without an assignment (there's no need for that)
    model_1_ml_.predict(
        days_in_season= 529,    # Num of days between 15/05/20 - 25/10/21
        shifting_12_days= True
    )
    # Updating the vectors to be able to run the model furthermore:
    model_1_ml_.update({
        'alpha_variant_vec':  neutralized_alpha_variant_vec,
        'delta_variant_vec': neutralized_delta_variant_vec,
        'isolation_morbidity_ratio_vector': neutralized_isolation_morbidity_ratio_vector,
        'is_lockdown': neutralized_lockdown_vec,
        'is_school': neutralized_school_vec,
        'v2_to_v3_transition_t' : neutralized_transition_vector,
        's_2_to_v3_transition_t' : neutralized_transition_vector
    })
    return model_1_ml_

# Update model for current variables of uncertainty:
def update_models_object(mdl_1_ml: mdl.Model_behave, realization: tuple):
    new_rho = mdl_1_ml.rho * realization[2]
    mdl_1_ml.update({
        'booster_efficiency': realization[0],
        'rho': new_rho
    })

#### calculating the transferring proportion:

In [5]:
def calc_trans_prop(vaccination_pq_: list, curr_res_mdl_1_ml: dict, curr_inv_: float):
    # Initializing the accumulated amount used and the priority queue:
    curr_inventory_used = 0
    vaccination_pq_copy = vaccination_pq_.copy()
    # Saving the combinations that we will vaccinate:
    vaccinated_que_ = []
    # Iterating until we cross the prior inventory level:
    while (curr_inventory_used <= curr_inv_):
        # Getting the current combination of (age_group, risk) from the pq:
        current_risk_age = vaccination_pq_copy.pop(0)
        # Getting the correspondent reported/unreported ratio for the Scenario:
        reported_unreported = get_reported_unreported_ratio('Scenario2')
        # Getting V2_j,r, S2_j,r proportions:
        S_2_age_risk, V_2_age_risk, R_2_age_risk, R_1_age_risk = calc_comp_prop_for_age_risk('S_2', curr_res_mdl_1_ml,current_risk_age), \
                                                                 calc_comp_prop_for_age_risk('V_2', curr_res_mdl_1_ml,current_risk_age), \
                                                                 calc_comp_prop_for_age_risk('R_2', curr_res_mdl_1_ml,current_risk_age), \
                                                                 calc_comp_prop_for_age_risk('R_1', curr_res_mdl_1_ml,current_risk_age)
        # Calculating the formula for num of needed vaccines:
        count_vaccines_in_use = (((R_1_age_risk + R_2_age_risk) * reported_unreported) + S_2_age_risk + V_2_age_risk) * pop_israel
        # Updating the accumulated used vaccines:
        curr_inventory_used += count_vaccines_in_use
        # Updating the vaccinated_que:
        vaccinated_que_.append(current_risk_age)

    final_trans_prop = curr_inv_ / curr_inventory_used
    return final_trans_prop, vaccinated_que_

#### Vaccinating in booster dose:

In [6]:
### Vaccinating in booster (moving from S_2 and V_2 to V_3, including the model update):
def vaccinate(trans_prop_: float, curr_res_mdl_1_ml: dict, vaccination_que: list):
    ## initialize settings:
    vaccination_que_copy = vaccination_que.copy()
    trans_prop_copy = trans_prop_
    # Getting the last day of the compartments and V_3, saving it to a dictionary:
    last_day_dict = {'S_2_last_day': np.reshape(curr_res_mdl_1_ml['S_2'][-1, :], newshape=(540) ),
                     'V_2_last_day': np.reshape(curr_res_mdl_1_ml['V_2'][-1, :], newshape=(540) ),
                     'V_3_last_day': np.reshape(curr_res_mdl_1_ml['V_3'][-1, :], newshape=(540) ) }
                     #'R_1_last_day': np.reshape(curr_res_mdl_1_ml['R_1'][-1, :], newshape=(540) ),
                     #'R_2_last_day': np.reshape(curr_res_mdl_1_ml['R_2'][-1, :], newshape=(540) )}

    # Iterating over the combinations of (risk, age_group) that we need to vaccinate, and moving the population, and updating the model:
    for curr_risk_age in vaccination_que_copy:
        # Getting the relevant indexes to operate in the correspondent locations of the compartment:
        curr_risk_age_indexes = get_indexes_risk_age_combination(curr_risk_age)

        ## Vaccinating from S_2 to V_3
        S_2_last_day_updated, V_3_last_day_updated = vaccinating_from_V_2_S_2(last_day_dict['V_3_last_day'], last_day_dict['S_2_last_day'], trans_prop_copy, curr_risk_age_indexes)
        # updating the results:
        last_day_dict['V_3_last_day'], last_day_dict['S_2_last_day'] = V_3_last_day_updated, S_2_last_day_updated

        ## Vaccinating from V_2 to V_3
        V_2_last_day_updated, V_3_last_day_updated = vaccinating_from_V_2_S_2(last_day_dict['V_3_last_day'], last_day_dict['V_2_last_day'], trans_prop_copy, curr_risk_age_indexes)
        # updating the results:
        last_day_dict['V_3_last_day'], last_day_dict['V_2_last_day'] = V_3_last_day_updated, V_2_last_day_updated

    update_compartment(curr_res_mdl_1_ml, 'V_2', last_day_dict['V_2_last_day'])
    update_compartment(curr_res_mdl_1_ml, 'S_2', last_day_dict['S_2_last_day'])
    update_compartment(curr_res_mdl_1_ml, 'V_3', last_day_dict['V_3_last_day'])

### Main - 1000 iterations of the Sim:

In [7]:
list_of_tuples

[(0.54, 0.2, 1.09, 7),
 (0.4, 0.11, 0.82, 9),
 (0.69, 0.17, 0.81, 10),
 (0.84, 0.12, 0.87, 4),
 (0.5, 0.15, 0.97, 5),
 (0.7, 0.11, 0.92, 5),
 (0.6, 0.18, 0.88, 7),
 (0.69, 0.1, 1.04, 4),
 (0.34, 0.2, 1.19, 9),
 (0.5, 0.11, 1.07, 6),
 (0.38, 0.15, 0.81, 10),
 (0.47, 0.17, 0.92, 7),
 (0.66, 0.12, 1.19, 9),
 (0.92, 0.19, 1.04, 10),
 (0.35, 0.12, 0.82, 5),
 (0.55, 0.12, 1.13, 5),
 (0.48, 0.15, 0.86, 9),
 (0.34, 0.2, 1.11, 4),
 (0.3, 0.18, 1.08, 8),
 (0.8, 0.1, 0.94, 3),
 (0.86, 0.16, 0.93, 3),
 (0.5, 0.13, 1.09, 8),
 (0.88, 0.15, 0.85, 8),
 (0.8, 0.16, 1.11, 6),
 (0.64, 0.14, 0.81, 3),
 (0.32, 0.17, 0.93, 7),
 (0.89, 0.12, 0.96, 9),
 (0.45, 0.1, 0.92, 4),
 (0.91, 0.18, 1.05, 9),
 (0.83, 0.12, 1.16, 7),
 (0.83, 0.19, 0.93, 3),
 (0.45, 0.14, 1.13, 9),
 (0.3, 0.15, 0.97, 4),
 (0.37, 0.13, 1.18, 5),
 (0.64, 0.17, 0.95, 10),
 (0.93, 0.12, 1.0, 5),
 (0.48, 0.1, 1.04, 7),
 (0.33, 0.13, 1.16, 4),
 (0.39, 0.15, 1.19, 4),
 (0.74, 0.18, 0.9, 8),
 (0.54, 0.16, 1.05, 7),
 (0.35, 0.19, 0.93, 4),
 (0.32,

In [8]:
# A list to hold 1000 realizations of cases per year:
cases_per_year_list = []

# Big loop run:
for tup in tqdm.tqdm(list_of_tuples, desc= 'tqdm() Progress Bar'):
    # Initializing the model's object (and its predictions) to t=0:
    model_1_ml = defining_model('Scenario2')
    # Settings:
    update_models_object(model_1_ml, tup)       # Updating the v3_efficiency and the proba for hosp
    inv_level = tup[1] * pop_israel             # Updating the inventory level
    years_for_model_run = tup[3]                # Updating the years for the model to run for

    # Running the model for X years, in 6 months resolution
    for i in range(years_for_model_run*2):
        # Running until the vaccination month:
        for j in range(1):
            res_mdl_1_ml = model_1_ml.predict(
                            days_in_season= 30,
                            continuous_predict= True
                            )
            # Getting the proportion of transition, and the combinations that we vaccinate:
            trans_prop, vaccinated_que = calc_trans_prop(vaccination_pq, res_mdl_1_ml, inv_level)
            # Vaccinating (including the model update):
            vaccinate(trans_prop, res_mdl_1_ml, vaccinated_que)

        # Running the model until the end of the current "single" run (half a year)
        for k in range(5):
            res_mdl_1_ml = model_1_ml.predict(
                days_in_season= 30,
                continuous_predict= True
            )
    ## Calculating the cases per year index for every measurement:
    tot_new_Is = np.add(res_mdl_1_ml['new_Is_1'].sum(axis=1) * pop_israel, res_mdl_1_ml['new_Is_2'].sum(axis=1) * pop_israel)
    tot_new_H = np.add(res_mdl_1_ml['new_H_1'].sum(axis=1) * pop_israel, res_mdl_1_ml['new_H_2'].sum(axis=1) * pop_israel)
    # Transferring to cumulative sum:
    cumsum_tot_new_Is, cumsum_tot_new_H = np.cumsum(tot_new_Is), np.cumsum(tot_new_H)
    # Performing the calculation itself:
    cases_per_year_new_Is, cases_per_year_new_H = (cumsum_tot_new_Is[-1] - cumsum_tot_new_Is[528]) / years_for_model_run, (cumsum_tot_new_H[-1] - cumsum_tot_new_H[528]) / years_for_model_run
    cases_per_year_list.append((cases_per_year_new_Is, cases_per_year_new_H))

tqdm() Progress Bar: 100%|██████████| 1000/1000 [4:21:13<00:00, 15.67s/it] 


In [9]:
cases_per_year_list

[(1543028.9414805386, 11117.500125115612),
 (1551231.1318346597, 8061.343114311358),
 (1536057.8175243686, 7752.156906729931),
 (1499515.3836034643, 9609.499859971516),
 (1545789.487200645, 10560.938103655268),
 (1524883.4736368705, 9880.27887015523),
 (1538633.0069152545, 8949.057582386904),
 (1521097.810511582, 11648.812458374165),
 (1554255.2298848548, 11723.793206416607),
 (1545824.511224483, 11248.033804181725),
 (1551640.3802573276, 7837.2983596278555),
 (1547960.926287366, 9414.990065006943),
 (1536757.5577432597, 11581.939778101772),
 (1522447.1427510655, 9861.247440468174),
 (1559885.352227031, 9011.636170258509),
 (1540794.694941293, 12262.566206431495),
 (1547018.1166281423, 8429.667458077058),
 (1564454.2081331122, 12789.340104968183),
 (1557466.7363493026, 10870.87616466534),
 (1491051.1647507763, 10936.810197931532),
 (1479199.8251573518, 10737.09861688522),
 (1545906.6866052502, 10883.78771904538),
 (1519668.7066109525, 8338.315234895766),
 (1518891.9840851931, 11462.627

In [10]:
with open('cases_per_year_list.pickle', 'wb') as handle:
    pickle.dump(cases_per_year_list, handle, protocol=pickle.HIGHEST_PROTOCOL)